In [3]:
import pandas as pd
import numpy as np
import random as rnd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
train_df = pd.read_csv('C:/Users/maste/PycharmProjects/Kaggle/Titanic/Dataset/train.csv')
test_df = pd.read_csv('C:/Users/maste/PycharmProjects/Kaggle/Titanic/Dataset/test.csv')
combine = [train_df, test_df]